In [5]:
import requests
from bs4 import BeautifulSoup
import pandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import datetime

# 제주도

In [71]:
def crawl_weather_data(year, month, date, time) :
    address =f'https://www.weather.go.kr/w/obs-climate/land/city-obs.do?auto_man=m&stn=0&dtm=&type=t99&reg=184&tm={year}.{month}.{date}.{time}%3A00'
    source = requests.get(address)
    soup = BeautifulSoup(source.content, 'html.parser')
    
    div = soup.find('div',{'class':'over-scroll'})

    weather = []
    for table in div.find_all('table'):
        tbodys = list(table.find_all('tbody'))
        
        for tbody in tbodys :
            tds = list(tbody.find_all('td'))
            
            for td in tbody.find_all('td') :
                if td.find('a') :
                    weather_dic = {}
                    point = td.find('a').text
                    # print('point>>>>>>>',point)
                    current_temp = tds[5].text
                    # print('current_temp>>>>>>>',current_temp)
                    body_temp = tds[7].text
                    # print('body_temp>>>>>>',body_temp)
                    weather_dic['point'] = point 
                    weather_dic['cur_temp'] = current_temp
                    weather_dic['body_temp'] = body_temp     
                    weather.append(weather_dic) 
                    
    weather_df = pd.DataFrame(weather)
    
    return weather_df

In [73]:
crawl_weather_data('2019','09','01','10')

,point,cur_temp,body_temp
0,고산,25.1,27.0
1,서귀포,25.1,27.0
2,성산,25.1,27.0
3,제주,25.1,27.0


In [6]:
pwd

'/Users/richardpark/AISchool/ml/socar_greecar_ml'

# 서울

In [11]:
current = datetime.datetime.now().date()
time = datetime.datetime.now().time()
print(current)
print(time)
print(time.hour)

2022-06-28
15:13:46.855109
15


In [38]:
def crawl_seoul_weather_data(address) :
    source = requests.get(address)
    soup = BeautifulSoup(source.content, 'html.parser')
    
    table = soup.find('table',{'border':'0'})
    
    weather_seoul = []
    for tr in table.find_all('tr') :
        tds = list(tr.find_all('td'))
        if len(tds) == 20 :
            point = tds[19].text
            # print('point >>>>>>',point)
            
            proto = []
            for td in tds :
                proto.append(td.text)
            
            weather_dic_seoul = {}
            weather_dic_seoul['point'] = proto[1]
            weather_dic_seoul['main_point'] = proto[19]
            weather_dic_seoul['temp'] = proto[10]
            weather_dic_seoul['rain'] = proto[3]
            weather_seoul.append(weather_dic_seoul)
            
    
    current = datetime.datetime.now().date()
    time = datetime.datetime.now().time().hour
    weather_seoul_df = pd.DataFrame(weather_seoul)
    
    # 경기도 삭제, 현충원, 기상청 삭제
    mask_g = weather_seoul_df['main_point'].str.contains('경기도')
    weather_seoul_df = weather_seoul_df[~mask_g]

    
    # 구 붙여주기
    for i in range(len(weather_seoul_df)) :
        gu = weather_seoul_df.iloc[i,1].split()[1]
        weather_seoul_df.iloc[i,0] = gu
    
    weather_seoul_df.to_csv(f'./{current}_{time}_기상청_비_온도_박대희_데이터.csv', index=False)     
    return weather_seoul_df

In [39]:
address = 'https://www.weather.go.kr/cgi-bin/aws/nph-aws_txt_min_cal_test?0&0&MINDB_01M&4108&a&M'
crawl_seoul_weather_data(address)

,point,main_point,temp,rain
0,종로구,서울특별시 종로구 송월동,26.6,○
1,중구,서울특별시 중구 예장동,24.5,○
2,용산구,서울특별시 용산구 이촌동,27.6,○
3,성동구,서울특별시 성동구 성수동1가,28.0,○
4,광진구,서울특별시 광진구 자양동,26.8,○
5,동대문구,서울특별시 동대문구 전농동,27.1,○
6,중랑구,서울특별시 중랑구 면목동,27.7,○
7,성북구,서울특별시 성북구 정릉동,25.8,○
8,강북구,서울특별시 강북구 수유동,27.0,○
9,도봉구,서울특별시 도봉구 방학동,26.3,○


In [40]:
weather_df = pd.read_csv('./2022-06-28_15_기상청_비_온도_박대희_데이터.csv')
weather_df

,point,main_point,temp,rain
0,종로구,서울특별시 종로구 송월동,26.6,○
1,중구,서울특별시 중구 예장동,24.5,○
2,용산구,서울특별시 용산구 이촌동,27.6,○
3,성동구,서울특별시 성동구 성수동1가,28.0,○
4,광진구,서울특별시 광진구 자양동,26.8,○
5,동대문구,서울특별시 동대문구 전농동,27.1,○
6,중랑구,서울특별시 중랑구 면목동,27.7,○
7,성북구,서울특별시 성북구 정릉동,25.8,○
8,강북구,서울특별시 강북구 수유동,27.0,○
9,도봉구,서울특별시 도봉구 방학동,26.3,○
